## Metatranscriptome data analysis pipeline: 
## go from raw counts to compiled data for downstream plotting


In [1]:
import pandas as pd
import pickle as cpk
import matplotlib as mpl
import matplotlib.pyplot as plt

## Before running this, run "gather-counts.py" and "SPOT_MetaT_allcompiled03232017.R" to (a) generate count files from salmon output and then compile all together using R. (b) R script saves allas "RawCounts_byContigs.csv"


In [2]:
#Parse orthoMCL file
##Contigs > 300bps were put through geneMark and then orthoMCL
##Result file: "results.clstr" needs to be parsed so we can 
##grab the contig IDs associated with each ortholog ID

def read_in_orthoMCL(infile):
    #uses ortholog cluster result and creates dictionary so that contigs are values assigned to ortholog key. 
    #Reads in results.clstr file from OrthoMCL output line by line
    cluster_dict={}
    with open(infile) as fi:
        for line in fi:
            if line.startswith(">"):
                line=line.strip()
                line=line.strip(">")
                cluster=line #cluster is now equal to the cluster name
                cluster_dict[cluster]=set() #set cluster as key in dictionary
            else:
                line=line.split(", ")[1]
                line=line.split("...")[0]
                contigID=line.strip(">") #isolates contig ID from non-cluster lines
                cluster_dict[cluster].add(contigID) #assigns as values in dictionary
    return(cluster_dict)

def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
 

In [3]:
#Parse the orthoMCL output to get a dictionary of ortho groups and contig ids

#Current location: /galadriel/sarah/metatranscriptome_orthomcl_dir/results.clstr

results_cluster_dict=read_in_orthoMCL("/galadriel/sarah/metatranscriptome_orthomcl_dir/results.clstr")
# cpk.dump(results_cluster_dict, open('results.cluster.dict.pickle', 'wb'))

In [7]:
results_cluster_dict
#This is a dictionary, where 'Cluster X' is the key and the contigIDs are the values

{'Cluster 0': {'k99_1022425_deep890m',
  'k99_1240574_deep150m',
  'k99_1470259_deep890m',
  'k99_1470565_deep890m',
  'k99_1592520_surface',
  'k99_1869874_deep150m',
  'k99_198107_deep890m',
  'k99_201743_deep890m',
  'k99_2075010_deep890m',
  'k99_2113549_deep890m',
  'k99_2248572_deep150m',
  'k99_2388889_deep890m',
  'k99_2514100_deep890m',
  'k99_2621771_deep890m',
  'k99_2636213_deep890m',
  'k99_288386_deep890m',
  'k99_296712_deep150m',
  'k99_3072323_deep150m',
  'k99_3094684_deep150m',
  'k99_312941_deep150m',
  'k99_3259690_deep150m',
  'k99_3407233_deep150m',
  'k99_34425_deep890m',
  'k99_362769_deep150m',
  'k99_3646076_deep150m',
  'k99_41632_deep890m',
  'k99_430285_deep150m',
  'k99_576502_deep150m',
  'k99_626266_deep890m',
  'k99_670931_deep150m',
  'k99_71933_deep890m',
  'k99_757296_deep890m',
  'k99_828492_deep890m',
  'k99_943722_deep890m'},
 'Cluster 1': {'k99_1048697_deep890m',
  'k99_1101111_deep150m',
  'k99_1101598_deep890m',
  'k99_1154896_deep150m',
  'k9

In [5]:
#Contigs were annotated with taxonomic IDs and Kegg IDs, import here ("annot_bycontigID.txt)
#Contig Key dataframe from the key values generated through annotation
contigKey_df=pd.read_csv("contigID_key_depth.txt", sep="\t") #read in annotation information for each contig_ID
contigKey_df=contigKey_df.set_index('contig_ID') #assign contig_ID as the index

In [6]:
contigKey_df.head()

,K0_number,Level2,Depth
contig_ID,,,
k99_10000_surface,K13789,Haptophyte,surface
k99_1000000_surface,NaN,NaN,surface
k99_1000002_surface,NaN,NaN,surface
k99_1000006_surface,NaN,NaN,surface
k99_1000007_surface,K13025,Chlorophyte,surface


## Read in raw counts (output from R)
Make a matrix based on the contig name and sample

In [7]:
#Read in raw counts to pd dataframe
#raw_counts_other = pd.read_csv('/galadriel/sarah/metatranscriptome_orthomcl_dir/raw_counts_byContig.txt', sep=' ')
#4/17/2017 - generated counts with associated contig IDs and samples.
raw_counts = pd.read_csv('RawCounts_byContigs.csv') #, sep=' '
raw_counts_flipped=raw_counts.set_index(['contig_ID', 'variable']).unstack() #set contig_ids as index
raw_counts_flipped.columns = raw_counts_flipped.columns.droplevel() #

In [8]:
raw_counts_flipped.head()

variable,Sample10,Sample11,Sample12,Sample25,Sample26,Sample27,Sample28,Sample29,Sample30,Sample31,Sample7,Sample8,Sample9
contig_ID,,,,,,,,,,,,,
k99_1000000_deep150m,0,0,0,2,3,2,0,0,0,0,0,0,0
k99_1000000_deep890m,0,0,0,0,0,0,19,56,34,54,0,0,0
k99_1000000_surface,1,2,0,0,0,0,0,0,0,0,0,0,0
k99_1000001_deep150m,0,0,0,3,4,0,0,0,0,0,0,0,0
k99_1000001_deep890m,0,0,0,4,0,7,3,7,12,7,0,0,0


## Goal: generate cluster information by taxa and by depth to get a better idea of the general composition of Orthologus Groupings
1) Identifying the Cluster that have a contig identified for each of the taxonomic groupings

In [9]:
# We are inverting the dictionary to get a dictionary 
##with dict[contig]=cluster ID and then generating a new DF
results_cluster_dict_invt=invert(results_cluster_dict)
contig_cluster_df=pd.DataFrame.from_dict(results_cluster_dict_invt,orient="index")#assign as index
contig_cluster_df.columns=['Cluster'] #called Cluster

In [10]:
contig_cluster_df.head()

,Cluster
k99_430285_deep150m,Cluster 0
k99_198107_deep890m,Cluster 0
k99_312941_deep150m,Cluster 0
k99_576502_deep150m,Cluster 0
k99_41632_deep890m,Cluster 0


In [11]:
# Dropped the values from contig_cluster_df that did not have orthologus groups. 
## This was due to a length cut off prior to the ortho clustering, 
## these contigs did not have associated protein sequences from genemark
contig_key_ortho_merged = contigKey_df.merge(contig_cluster_df, left_index=True, right_index=True)
contig_key_ortho_merged.Level2.fillna("Unannotated", inplace=True)
contig_key_ortho_merged['Count']=0 #dummy count column

In [12]:
contig_key_ortho_merged.head() #merged contig ID, annotation information, and ortholog information
#merged with raw counts further below

,K0_number,Level2,Depth,Cluster,Count
k99_430285_deep150m,K01803,Unannotated,deep150m,Cluster 0,0
k99_198107_deep890m,K06805,Unannotated,deep890m,Cluster 0,0
k99_312941_deep150m,NaN,Unannotated,deep150m,Cluster 0,0
k99_576502_deep150m,NaN,Dinoflagellate,deep150m,Cluster 0,0
k99_41632_deep890m,NaN,Unannotated,deep890m,Cluster 0,0


In [13]:
Ortho_by_Taxa = contig_key_ortho_merged.groupby(['Cluster', 'Level2']).count()['Count'].unstack()
#apply a groupby with an ortholog cluster; group contigs within an ortholog by occurences of taxa
Ortho_by_Taxa.fillna(0, inplace=True)
Ortho_by_Taxa.to_csv('Ortho_groups_by_taxa.csv')
# Ortho_by_Taxa.head() #number of contigs (occurences) belonging to each taxonomic group within a cluster

In [25]:
Ortho_by_Taxa_Binary=Ortho_by_Taxa.copy()
Ortho_by_Taxa_Binary[Ortho_by_Taxa_Binary>0]=1
Ortho_by_Taxa_Binary.to_csv('Ortho_groups_by_taxa_binary.csv')
# Use this output to generate "upsetR" plot to show distribution of taxa at all depths

In [42]:
# Ortho_by_Taxa.head() #Count of how many taxonomic groups are in each cluster
# Ortho_by_Taxa_Binary.head() #Binary version, this one is used in upsetR plot, because we are graphing presence of taxonomic group in each ortholog group

In [26]:
#Identifying the core contigs that are shared by all five taxonomic groups #index list
##Using binary file, as long as sum is 5 across those groups of interest, it is shared
Core_Contigs_Taxa=Ortho_by_Taxa_Binary[Ortho_by_Taxa_Binary.iloc[:,0:5].sum(axis=1)==5].index

In [27]:
Core_Ortho_By_taxa_Cotigs=contig_key_ortho_merged[contig_key_ortho_merged.Cluster.isin(list(Core_Contigs_Taxa))]


In [28]:
Core_Ortho_By_taxa_Cotigs.head()

,K0_number,Level2,Depth,Cluster,Count
k99_1477958_surface,K10408,Diatom,surface,Cluster 19,0
k99_2476140_deep150m,K10408,Uncertain,deep150m,Cluster 19,0
k99_187379_deep150m,K10408,Other,deep150m,Cluster 19,0
k99_1891267_surface,K10408,Uncertain,surface,Cluster 19,0
k99_2378859_deep150m,K10408,Uncertain,deep150m,Cluster 19,0


In [29]:
#Subset only the identified 'core' orthologus groups
Core_Ortho_By_taxa_Cotigs=contig_key_ortho_merged[contig_key_ortho_merged.Cluster.isin(list(Core_Contigs_Taxa))]
#Merge with raw counts - by index
Core_Ortho_By_taxa_Cotigs_reads=Core_Ortho_By_taxa_Cotigs.merge(raw_counts_flipped, left_index=True, right_index=True)
#Selects taxa of interest
Core_Ortho_By_taxa_Cotigs_reads=Core_Ortho_By_taxa_Cotigs_reads[Core_Ortho_By_taxa_Cotigs_reads.Level2.isin(['Chlorophyte','Ciliate','Diatom',
                                                                   'Dinoflagellate','Haptophyte',])]
#Group those orthologs by taxa ID, & replace NaN with zeros
Core_Ortho_By_taxa_Cotigs_reads=Core_Ortho_By_taxa_Cotigs_reads.groupby(['Cluster',
                                                                         'Level2']).sum().fillna(0).T.drop('Count').T
#Core_Ortho_By_taxa_Cotigs_reads.to_csv('Raw_Counts_ByTaxa_CommonOG.csv')


In [32]:
len(Core_Ortho_By_taxa_Cotigs_reads) #974 - 4/17/2017
#Core_Ortho_By_taxa_Cotigs_reads.head()

974

## 1a) Running the clustering data by depth instead of by taxa

In [33]:
contig_key_ortho_merged.head() #Contig IDs, with K0, tax, & ortholog cluster information 


,K0_number,Level2,Depth,Cluster,Count
k99_312941_deep150m,NaN,Unannotated,deep150m,Cluster 0,0
k99_2248572_deep150m,NaN,Unannotated,deep150m,Cluster 0,0
k99_1470259_deep890m,NaN,Unannotated,deep890m,Cluster 0,0
k99_1022425_deep890m,NaN,Unannotated,deep890m,Cluster 0,0
k99_41632_deep890m,NaN,Unannotated,deep890m,Cluster 0,0


In [34]:
Ortho_by_depth = contig_key_ortho_merged.groupby(['Cluster', 'Depth']).count()['Count'].unstack()
Ortho_by_depth.fillna(0, inplace=True)


In [44]:
Ortho_by_depth.head()

Depth,deep150m,deep890m,surface
Cluster,,,
Cluster 0,14.0,19.0,1.0
Cluster 1,19.0,19.0,0.0
Cluster 10,7.0,1.0,11.0
Cluster 100,0.0,1.0,0.0
Cluster 1000,1.0,16.0,0.0


In [45]:
Ortho_by_depth_Binary=Ortho_by_depth.copy()
Ortho_by_depth_Binary[Ortho_by_depth_Binary>0]=1
Ortho_by_depth_Binary.to_csv('Ortho_by_depth_Binary.csv')
#Use Ortho_by_depth_Binary to plot UpsetR plot
#Distribution of contigs among ortholog groups by depth
#If contigs within an ortholog appeared at a given depth, assigned "1"

In [46]:
#Orthologs listed with how many contigs had Kegg IDs at all depths
Ortho_by_depth = contig_key_ortho_merged.groupby(['Cluster', 'Depth']).count()['K0_number'].unstack()


In [47]:
Ortho_by_depth

Depth,deep150m,deep890m,surface
Cluster,,,
Cluster 0,1.0,2.0,0.0
Cluster 1,2.0,2.0,NaN
Cluster 10,0.0,0.0,5.0
Cluster 100,NaN,0.0,NaN
Cluster 1000,0.0,0.0,NaN
Cluster 10000,NaN,4.0,NaN
Cluster 100000,NaN,1.0,NaN
Cluster 1000000,NaN,NaN,1.0
Cluster 1000001,NaN,0.0,0.0


In [48]:
#Identifying the KOs that are annotated with some sort KID

Annotated_withKO_ids=list(Ortho_by_depth[pd.DataFrame.any(Ortho_by_depth>0, axis=1)].index)

outFile=open('Annotated_withKO_ids.txt', 'w')
for ko in Annotated_withKO_ids:
    outFile.write(ko)
    outFile.write('\n')
outFile.close()

#Use .txt file in UpsetR plot to designate contigs with assigned Kegg IDs

## 2) Identifying the KO groupings that are present in each of the five taxonomic groups and then summing the reads by the KO groupings.

In [49]:
#Identify the KO groups that are common by taxonomic group across the five groups

ko_taxa_group=contig_key_ortho_merged.groupby(['K0_number', 'Level2']).count()['Count'].unstack()
common_kos=list(ko_taxa_group.iloc[:,0:5].dropna().index) #drop rows with NAs of the first 5
#get list of K0 IDs that were found in at least 1 taxonomic group

#print out list with the common taxa
outfile=open('Core_KO_Taxa.txt', 'w')
for ko in common_kos:
    outfile.write(ko)
    outfile.write('\n')
outfile.close()

In [39]:
#Get the counts for the common kos and group them by the groups of interest

common_ko_contigs=contig_key_ortho_merged[contig_key_ortho_merged.K0_number.isin(common_kos)]
common_ko_contigs=common_ko_contigs[common_ko_contigs.Level2.isin(['Chlorophyte','Ciliate','Diatom',
                                                                   'Dinoflagellate','Haptophyte',])]

#merge reads with counts and sum reads by KO and taxonomic group
raw_counts_common_ko=common_ko_contigs.merge(raw_counts_flipped, left_index=True, right_index=True)
raw_counts_common_ko=raw_counts_common_ko.groupby(['K0_number', 'Level2']).sum().fillna(0).T.drop('Count').T
raw_counts_common_ko.to_csv('Raw_Counts_ByTaxa_CommonKO.csv')

In [40]:
#len(common_ko_contigs)

462897

##3) Create raw count table for input into edgeR

In [41]:
#Merge contig key by ortholog with raw counts
#will update counts as soon as new count information from salmon run is complete - 3/23/2017
raw_counts_bycontig_byortholog=contig_key_ortho_merged.merge(raw_counts_flipped, left_index=True, right_index=True)


In [42]:
#all counts associated with contigs and ortholog clusters, included K0 and tax IDs
raw_counts_bycontig_byortholog.head()
#save, normalize in EdgeR for all plots
raw_counts_bycontig_byortholog.to_csv('RawCounts_bycontig_annotated.csv') #previous name:RawCounts_all_bycontig.csv